# Bootcamp: Cientista de Dados - Desafio

- Data: junho de 2022.

## Autor

Feito com por [Alexsander Lopes Camargos](https://github.com/alexcamargos): Entre em contato!

[![GitHub](https://img.shields.io/badge/-AlexCamargos-1ca0f1?style=flat-square&labelColor=1ca0f1&logo=github&logoColor=white&link=https://github.com/alexcamargos)](https://github.com/alexcamargos)
[![Twitter Badge](https://img.shields.io/badge/-@alcamargos-1ca0f1?style=flat-square&labelColor=1ca0f1&logo=twitter&logoColor=white&link=https://twitter.com/alcamargos)](https://twitter.com/alcamargos)
[![Linkedin Badge](https://img.shields.io/badge/-alexcamargos-1ca0f1?style=flat-square&logo=Linkedin&logoColor=white&link=https://www.linkedin.com/in/alexcamargos/)](https://www.linkedin.com/in/alexcamargos/)
[![Gmail Badge](https://img.shields.io/badge/-alcamargos@vivaldi.net-1ca0f1?style=flat-square&labelColor=1ca0f1&logo=Gmail&logoColor=white&link=mailto:alcamargos@vivaldi.net)](mailto:alcamargos@vivaldi.net)

## Licença

[MIT License](https://choosealicense.com/licenses/mit/)

# Módulo 2 - Desenvolvimento de Soluções Utilizando Spark

## Objetivos

Exercitar os seguintes conceitos trabalhados no Módulo:

- Exercitar o módulo Spark SQL do Apache Spark.
- Exercitar o módulo Spark MLLib do Apache Spark.

## Enunciado

Doenças ligadas ao coração afetam milhões de pessoas ao redor do mundo, e segundo a Organização Mundial de Saúde (OMS) é a segunda principal causa de morte na população mundial. Como cientista de dados, você foi contratado para criar um modelo preditivo que, a partir de dados de pacientes - como idade, gênero, nível de glicose, se é fumante ou não - vai prever se eles terão um derrame cerebral ou não.

Você tem acesso a um arquivo que possui atributos de pacientes e um atributo “stroke” (derrame), que indica se aquele paciente sofreu um evento de derrame ou não.

O conjunto de dados está disponível em: [https://dcc.ufmg.br/~pcalais/stroke_data.csv](https://dcc.ufmg.br/~pcalais/stroke_data.csv)

Para uma descrição das colunas, veja a seção “Attribute information” em [https://www.kaggle.com/fedesoriano/stroke-prediction-dataset](https://www.kaggle.com/fedesoriano/stroke-prediction-dataset).

As questões objetivas vão lhe guiar sobre a análise exploratória e o modelo preditivo que você criará a partir dos dados.

### Links úteis:
- [https://spark.apache.org/docs/latest/sql-getting-started.html](https://spark.apache.org/docs/latest/sql-getting-started.html)
- [https://spark.apache.org/docs/latest/ml-classificationregression.html#decision-tree-classifier](https://spark.apache.org/docs/latest/ml-classificationregression.html#decision-tree-classifier)

## Descrição das colunas

- *id*: unique identifier
- *gender*: "Male", "Female" or "Other"
- *age*: age of the patient
- *hypertension*: 0 if the patient doesn't have hypertension, 1 if the patient has hypertension
- *heart_disease*: 0 if the patient doesn't have any heart diseases, 1 if the patient has a heart disease
- *ever_married*: "No" or "Yes"
- *work_type*: "children", "Govt_jov", "Never_worked", "Private" or "Self-employed"
- *Residence_type*: "Rural" or "Urban"
- *avg_glucose_level*: average glucose level in blood
- *bmi*: body mass index
- *smoking_status*: "formerly smoked", "never smoked", "smokes" or "Unknown"*
- *stroke*: 1 if the patient had a stroke or 0 if not

_*Note: "Unknown" in smoking_status means that the information is unavailable for this patient_

# Pergunta 1

Quantos registros existem no arquivo?

In [0]:
pass

# Pergunta 2

*Quantas colunas existem no arquivo? Quantas são numéricas?*

Ao ler o arquivo com spark.read.csv, habilite `inferSchema=True`. Use a função `printSchema()` da API de Dataframes.

In [0]:
pass

# Pergunta 3

*No conjunto de dados, quantos pacientes sofreram e não sofreram derrame (stroke), respectivamente?*

In [0]:
pass

# Pergunta 4

A partir do dataframe, crie uma tabela temporária usando `df.createOrReplaceTempView('table')` e a seguir use `spark.sql` para escrever uma consulta SQL que obtenha quantos pacientes tiveram derrame por tipo de trabalho (_work_type_).

*Quantos pacientes sofreram derrame e trabalhavam respectivamente, no setor privado, de forma independente, no governo e quantas são crianças?*

In [0]:
pass

# Pergunta 5

Escreva uma consulta com `spark.sql` para determinar a proporção, por gênero, de participantes do estudo. A maioria dos participantes é:

In [0]:
pass

# Pergunta 6

Escreva uma consulta com `spark.sql` para determinar quem tem mais probabilidade de sofrer derrame: hipertensos ou não-hipertensos. Você pode escrever uma consulta para cada grupo. A partir das probabilidades que você obteve, você conclui que:

In [0]:
pass

# Pergunta 7

*Escreva uma consulta com `spark.sql` que determine o número de pessoas que sofreram derrame por idade. Com qual idade o maior número de pessoas do conjunto de dados sofreu derrame?*

In [0]:
pass

# Pergunta 8

Usando a API de dataframes, determine quantas pessoas sofreram derrames após os 50 anos.

In [0]:
pass

# Pergunta 9

Usando `spark.sql`, determine qual o nível médio de glicose para pessoas que, respectivamente, sofreram e não sofreram derrame.

In [0]:
pass

# Pergunta 10

*Qual é o _BMI (IMC = índice de massa corpórea)_ médio de quem sofreu e não sofreu derrame?*

In [0]:
pass

# Pergunta 11

Crie um modelo de árvore de decisão que prevê a chance de derrame (stroke) a partir das variáveis contínuas/categóricas: idade, BMI, hipertensão, doença do coração, nível médio de glicose.

_Use o conteúdo da segunda aula interativa para criar e avaliar o modelo._

In [0]:
pass

# Pergunta 12

Adicione ao modelo as variáveis categóricas: gênero e status de fumante. Use o conteúdo da aula interativa para lidar com as variáveis categóricas.  A acurácia (qualidade) do modelo aumentou para:

In [0]:
pass

#Pergunta 13

Qual dessas variáveis é mais importante no modelo de árvore de decisão que você construiu na questão (12)?

In [0]:
pass

# Pergunta 14

Qual a profundidade da árvore de decisão da questão (12)?

In [0]:
pass

# Pergunta 15

Quantos nodos a árvore de decisão possui?

In [0]:
pass